# Human Population Size on an Earth-like Planet--a Computer Experiment

In [191]:
import sim_world_functions as swf

import importlib
from pathlib import Path
import sys

import networkx as nx
from pyvis.network import Network
import numpy as np
import polars as pl
import pyarrow
from scipy import stats
from pert import PERT
from shapely.geometry import Point
import geopandas
import plotly.express as px

In [192]:
importlib.reload(swf)
rng = np.random.default_rng()
pl.Config.set_fmt_str_lengths(n=100)
DATA_PATH = Path("../data/")

The response variable is the arithmetic mean of the normalized coefficients of variation for the population sizes.

$$
z_{i} = \frac{1}{T + 1}
\sum_{t=0}^{T}
    \frac{
        \sqrt{
            \hat{V} \left(
            \mathrm{logpopsize}_{i,t}
            \right)
        }
    }
    {
        \underset{j}{\mathrm{median}}\left(\mathrm{logpopsize}_{i,j,t}\right)
    } 
         
                
$$

where $i=1,2,\dots,k$ indexes the treatment, $j=1,2,\dots n_i$ indexes the replicate for the $i^{\text{th}}$ treatment, and $t=1,2,\dots,T$ indexes the time.

$\hat{V}$ denotes the unbiased sample variance which is taken of the natural logarithms of the population sizes observed for treatment $i$ at time $t$.

## Demographic Balancing Equation:

$$
x_{i,t} = b_{i, t-1} + \sum_{j}{m_{j, i, t-1}} - \sum_{j}{m_{i,j,t-1}} - d_{1, t-1}
$$

where

$$
\begin{aligned}
x_{i,t} &= \text{population size at location } i \text{ at time } t\\
b_{i, t-1} &= \text{number of births at location } i \text{ at time } t-1 \\
m_{j, i, t-1} &= \text{number of people immigrating to } i \text{ from } j \text{ at time } t-1 \\
m_{i, j, t-1} &= \text{number of people emigrating from } i \text{ to } j \text{ at time } t-1 \\
d_{1, t-1} &= \text{number of deaths at location } i \text{ at time } t-1
\end{aligned}
$$

In [3]:
# Constants for the Entire Experiment
SIMULATION_YEARS = 10

# Get Data from Other Scripts

In [4]:
# ETL for distance matrix:
dist_mat = np.loadtxt(Path(DATA_PATH, "dist_matrix.csv"), delimiter=",")
# Set num_world_locations according to pre-made
# dist_mat.
num_world_locations = dist_mat.shape[0]
# Manipulate dist_mat 
# https://stackoverflow.com/questions/16444930/copy-upper-triangle-to-lower-triangle-in-a-python-matrix
dist_mat = np.triu(dist_mat)
dist_mat = dist_mat + dist_mat.T
np.fill_diagonal(a=dist_mat, val=0)

## Get geographic data that goes along with the distance matrix.

In [5]:
# This takes 3 min on my computer.
# https://ecoregions.appspot.com/
ecoregions_2017_with_more_points = geopandas.read_file(filename=Path(DATA_PATH, "ecoregions_2017_with_more_points.shp"))

In [6]:
# Convert to an equal-area CRS so that we can find
# the areas of the ecoregions.
# https://gis.stackexchange.com/questions/285266/geopandas-proj4-reproject-to-global-equal-area-projection
ecoregions_2017_with_more_points.to_crs(crs="+proj=eck4 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +ellps=WGS84 +units=m +no_defs", inplace=True)
# Get the area of each ecoregion in square meters.
ecoregions_2017_with_more_points["ecoregion_area_in_sq_m"] = ecoregions_2017_with_more_points.area

In [71]:
# Polars is way better than pandas!!!
# Create a polars dataframe without the geometry info.
# for performance reasons.
ecoregions_2017_with_more_points_no_geo = pl.from_pandas(
    data=ecoregions_2017_with_more_points.loc[:, ["REALM", "BIOME_NAME", "ECO_NAME", "ecoregion_area_in_sq_m"]]
)

ecoregions_2017_with_more_points_no_geo = (ecoregions_2017_with_more_points_no_geo
    .sort(["REALM", "BIOME_NAME", "ECO_NAME"]) 
    .with_row_count(name="location")
    .with_columns([
        # Convert the area of each ecoregion to square miles.
        (pl.col("ecoregion_area_in_sq_m") / (1609.34**2)).alias("ecoregion_area_in_sq_mi"),

        # Cast datatype for join later
        pl.col("location").cast(pl.Int64).alias("location")
    ])  
    .drop("ecoregion_area_in_sq_m")                                        
)

In [72]:
ecoregions_2017_with_more_points_no_geo.tail()

location,REALM,BIOME_NAME,ECO_NAME,ecoregion_area_in_sq_mi
i64,str,str,str,f64
1465,"""Palearctic""","""Tundra""","""Trans-Baikal Bald Mountain tundra""",84199.773149
1466,"""Palearctic""","""Tundra""","""Wrangel Island Arctic desert""",2916.662085
1467,"""Palearctic""","""Tundra""","""Yamal-Gydan tundra""",159486.350881
1468,"""Palearctic""","""Tundra""","""Yamal-Gydan tundra""",159486.350881
1469,"""Palearctic""","""Tundra""","""Yamal-Gydan tundra""",159486.350881


In [9]:
ecoregions_2017_with_more_points_no_geo.drop("ecoregion_area_in_sq_mi").write_csv(file=Path(DATA_PATH, "ecos.tsv"), separator="\t")

Use ecoregions_2017_with_more_points_no_geo to store data about each location.

In [73]:
# What are the areas of each location
# if each ecoregion in which they are located
# is divided into equal area sections according
# to the number of locations in that ecoregion?
ecoregions_2017_with_more_points_no_geo = (ecoregions_2017_with_more_points_no_geo
    .with_columns([
        (pl.col("ecoregion_area_in_sq_mi") / pl.count("ECO_NAME").over("ECO_NAME"))
        .alias("location_area_in_sq_mi")
    ])
)

In [74]:
ecoregions_2017_with_more_points_no_geo.tail()

location,REALM,BIOME_NAME,ECO_NAME,ecoregion_area_in_sq_mi,location_area_in_sq_mi
i64,str,str,str,f64,f64
1465,"""Palearctic""","""Tundra""","""Trans-Baikal Bald Mountain tundra""",84199.773149,42099.886575
1466,"""Palearctic""","""Tundra""","""Wrangel Island Arctic desert""",2916.662085,2916.662085
1467,"""Palearctic""","""Tundra""","""Yamal-Gydan tundra""",159486.350881,53162.11696
1468,"""Palearctic""","""Tundra""","""Yamal-Gydan tundra""",159486.350881,53162.11696
1469,"""Palearctic""","""Tundra""","""Yamal-Gydan tundra""",159486.350881,53162.11696


# Get Planning Matrix

In [99]:
# This matrix was constructed in R.
planning_matrix = pl.read_csv(
    source=Path(DATA_PATH, "planning_matrix.txt"),
    has_header=True,
    separator=" "
)

In [100]:
planning_matrix.head()

earth,location,REALM,BIOME_NAME,ECO_NAME,initial_agricultural_tech_level_in_use,initial_healthcare_tech_level_in_use,initial_housing_tech_level_in_use,initial_transportation_tech_level_in_use,initial_unaided_d_max_pop_density,initial_unaided_not_dt_max_pop_density,initial_unaided_t_max_pop_density,initial_warfare_tech_level_in_use,relevance_of_dist_based_on_transportation_tech_level,initial_pop_density_of_desert,initial_pop_density_of_tundra,initial_proportion_not_desert,initial_pop_density_ratio,initial_transition_probability_to_desert
i64,i64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,0,"""Afrotropic""","""Deserts & Xeric Shrublands""","""Aldabra Island xeric scrub""",0.643977,0.000387,0.046324,0.161329,1.419058,6.43861,1.319,0.061229,1768.624322,0.220119,0.238211,0.88836,777.047765,0.004062
1,1,"""Afrotropic""","""Deserts & Xeric Shrublands""","""Djibouti xeric shrublands""",0.662884,0.08286,0.321868,0.341016,0.473389,8.203317,1.713027,0.027305,-1228.728912,0.220119,0.238211,0.88836,777.047765,0.004062
1,2,"""Afrotropic""","""Deserts & Xeric Shrublands""","""Eritrean coastal desert""",0.63524,0.021075,0.230545,0.153925,1.6928435,5.129825,1.0713665,0.087405,-609.0,0.220119,0.238211,0.88836,777.047765,0.004062
1,3,"""Afrotropic""","""Deserts & Xeric Shrublands""","""Gariep Karoo""",0.63524,0.021075,0.230545,0.153925,1.6928435,5.129825,1.0713665,0.087405,-609.0,0.220119,0.238211,0.88836,777.047765,0.004062
1,4,"""Afrotropic""","""Deserts & Xeric Shrublands""","""Gariep Karoo""",0.63524,0.021075,0.230545,0.153925,1.6928435,5.129825,1.0713665,0.087405,-609.0,0.220119,0.238211,0.88836,777.047765,0.004062


# Construct Complete Graph and Initialize it With Starting Attributes for Experiment

In [101]:
pre_world = nx.complete_graph(num_world_locations)
# We'll use this to filter now instead
# of having to repeatedly filter later.
what_earth_we_on = 1

In [102]:
# ed stands for earth data
ed = (planning_matrix
    .filter(pl.col("earth") == what_earth_we_on)
    # Drop them now because we'll pick them up again
    # via the join.
    .drop(["REALM",	"BIOME_NAME", "ECO_NAME"])
    .join(
        other=ecoregions_2017_with_more_points_no_geo,
        how="inner",
        on="location"
    )
)

In [198]:
##############################################
# Get ecoregions that were not in desert before.
##############################################

unique_ecos_within_deserts_within_realm = (ed
    .filter(pl.col("BIOME_NAME") == "Deserts & Xeric Shrublands")
    .select(
        pl.col("REALM"), 
        pl.col("ECO_NAME").alias("unique_ecos_within_deserts_within_realm")
    )
    .unique()
    .sort(by="REALM")
    # https://stackoverflow.com/a/70061932/8423001
    .with_columns([
        pl.lit(1).alias("ones_counter")
    ])
    .select(
        pl.all().exclude("ones_counter"),
        pl.col("ones_counter").cum_sum().over("REALM").alias("unique_ecos_within_deserts_within_realm_id")
    )
)


num_unique_ecos_within_deserts_within_realm = (ed
    .filter(pl.col("BIOME_NAME") == "Deserts & Xeric Shrublands")
    .select(
        pl.col("REALM"), 
        (pl.col("ECO_NAME").unique().count()).over("REALM").alias("num_unique_ecos_within_deserts_within_realm")
    )
    .unique()
)

u = (num_unique_ecos_within_deserts_within_realm
    .join(
        other=ed,
        on="REALM",
        how="inner"
    )
    .with_columns([
        (
            pl.col("num_unique_ecos_within_deserts_within_realm") * pl.col("initial_proportion_not_desert")
        )
        .round()
        .cast(pl.Int64)
        .alias("num_unique_ecos_within_deserts_within_realm_to_un_desert")
    ])
    .select([
        "REALM",
        "num_unique_ecos_within_deserts_within_realm",
        "num_unique_ecos_within_deserts_within_realm_to_un_desert"
    ])
    .unique(subset="REALM")
)

ecos_to_un_desert = pl.Series(name="ecos_to_un_desert", dtype=pl.Utf8)
for realm, num_unique_ecos_within_deserts_within_realm, num_unique_ecos_within_deserts_within_realm_to_un_desert in u.iter_rows():
    # These are the indices of unique ecoregions
    # to un-desert.
    indices_to_un_desert = rng.choice(
        a=np.arange(start=1, stop=num_unique_ecos_within_deserts_within_realm + 1, step=1), 
        size=num_unique_ecos_within_deserts_within_realm_to_un_desert,
        replace=False
    )
    
    indices_to_un_desert.sort()

    ecos_to_un_desert.append(unique_ecos_within_deserts_within_realm
        .filter(
            (pl.col("REALM") == realm)
            & (pl.col("unique_ecos_within_deserts_within_realm_id").is_in(indices_to_un_desert))
        )
        .select("unique_ecos_within_deserts_within_realm")
        .to_series()
    )

# Do the un-deserting.
prior_biomes = swf.prior_biomes_before_desertification(len(ecos_to_un_desert), rng)

desertification = pl.DataFrame(
    data={
        "eco_to_un_desert": ecos_to_un_desert,
        "prior_biome": prior_biomes
    }
)

In [199]:
# Make edge weights between the world locations
# to represent the distance between those locations.
# https://stackoverflow.com/questions/17051589/parsing-through-edges-in-networkx-graph
for (v1, v2, weight) in pre_world.edges.data('weight'):
    # https://trenton3983.github.io/files/projects/2020-05-21_intro_to_network_analysis_in_python/2020-05-21_intro_to_network_analysis_in_python.html
    # https://stackoverflow.com/questions/40128692/networkx-how-to-add-weights-to-an-existing-g-edges

    pre_world[v1][v2]["weight"] = dist_mat[v1, v2]

In [202]:
# Update initial node attributes
pre_world_node_attributes = {}
# Handle desertification first.
for node_id in range(num_world_locations):
    # Are we in a location within an ecoregion
    # that needs to be un-deserted?
    eco = ed[node_id, "ECO_NAME"]
    actual_biome = ed[node_id, "BIOME_NAME"]
    if eco in ecos_to_un_desert:
        # un-desert
        actual_biome = (desertification
            .filter(pl.col("eco_to_un_desert") == eco)
            .select("prior_biome")
            .item()
        )
    pre_world_node_attributes[node_id] = {
        # Are we in a location within an ecoregion
        # that needs to be un-deserted?
        "actual_biome": actual_biome
    }
# Set our node attributes.
# https://stackoverflow.com/questions/53508805/simple-way-for-modifying-attributes-of-single-nodes-in-networkx-2-1?rq=3
nx.set_node_attributes(pre_world, pre_world_node_attributes)

In [66]:
# Now, update other initial node attributes
for node_id in range(num_world_locations):
    # Set the pop_size attribute.
    if pre_world_node_attributes[node_id]["actual_biome"] == "Deserts & Xeric Shrublands":
        pre_world_node_attributes[node_id] = {
            "pop_size": round(cep[node_id, "initial_pop_density_of_desert"] \
                * ecoregions_2017_with_more_points_no_geo[node_id, "location_area_in_sq_mi"])
        }
    elif pre_world_node_attributes[node_id]["actual_biome"] == "Tundra":
        pre_world_node_attributes[node_id] = {
            "pop_size": round(cep[node_id, "initial_pop_density_of_tundra"] \
                * ecoregions_2017_with_more_points_no_geo[node_id, "location_area_in_sq_mi"]) 
        }
    else:
        # We are at a location where the pop_size depends
        # on the mean of initial_pop_density_of_desert and initial_pop_density_of_tundra.
        mean_uninhabitable_initial_pop_density = (
            cep[node_id, "initial_pop_density_of_desert"] \
                + cep[node_id, "initial_pop_density_of_tundra"]
            ) / 2
        
        pre_world_node_attributes[node_id] = {
            "pop_size": round(cep[node_id, "initial_pop_density_ratio"] \
                * mean_uninhabitable_initial_pop_density \
                * ecoregions_2017_with_more_points_no_geo[node_id, "location_area_in_sq_mi"])
        }
        
        # "pop_size": treatments[node_id, "initial_pop_size"], 
        # "carrying_capacity": treatments[node_id, "initial_unaided_carrying_cap"],
        # "transportation_technology_level_in_use": INITIAL_TRANSPORTATION_TECH_LEVEL_IN_USE.rvs().item(),
        # "sortino_ratio": 0.5,
        # # 0 = no knowledge
        # # 1 = perfect knowledge
        # "knowledge_of_neighbors": np.zeros(shape=num_world_locations),
        # # calculate this as a softargmax of sortino ratios 
        # "proportion_desirous_to_emigrate": 0.2,
        # "emigration_success_rate": 0.5,
        # "energy_hills_to_neighbors": np.zeros(shape=num_world_locations)
    # }
    # for neighbor in pre_world.neighbors(node_id):
    #     # Calculate stuff we need to store in the current 
    #     # node in relation to its neighbors.
    #     relevance_of_dist = swf.weight_0_more(
    #         x=pre_world_node_attributes[node_id]["transportation_technology_level_in_use"],
    #         b=RELEVANCE_OF_DIST_BASED_ON_TRANSPORTATION_TECH_LEVEL
    #     )
    #     energy_hill_to_neighbor = relevance_of_dist * pre_world[node_id][neighbor]["weight"] 
    #     # Add info. to attribute dict for node_id.
    #     pre_world_node_attributes[node_id]["energy_hills_to_neighbors"][neighbor] = energy_hill_to_neighbor

# Update our node attributes.
# https://stackoverflow.com/questions/53508805/simple-way-for-modifying-attributes-of-single-nodes-in-networkx-2-1?rq=3
nx.set_node_attributes(pre_world, pre_world_node_attributes)

In [68]:
pre_world.nodes[5]

{'actual_biome': 'Flooded Grasslands & Savannas', 'pop_size': 926693}

# Test Run

In [ ]:
# Test looping structure
# https://stackoverflow.com/questions/53508805/simple-way-for-modifying-attributes-of-single-nodes-in-networkx-2-1?rq=3
for t in range(SIMULATION_YEARS):
    for node_id in range(num_world_locations):
        carrying_cap = pre_world_node_attributes[node_id]["carrying_capacity"]
        pop_size = pre_world_node_attributes[node_id]["pop_size"]

        pre_world_node_attributes[node_id]["pop_size"] = swf.logistic_growth(
            previous_pop=pop_size,
            r=0.05,
            carrying_cap=carrying_cap
        )
        
        
# Set our node attributes.
nx.set_node_attributes(pre_world, pre_world_node_attributes)

In [ ]:
pre_world_betweenness_centralities = nx.betweenness_centrality(
    G=pre_world,
    weight="weight"
)

pre_biomes_betweenness_centralities = nx.betweenness_centrality(
    G=pre_biomes
)

# Get a node with a maximal betweenness centrality.
# This node will hold our starting population.
# https://stackoverflow.com/a/280156/8423001
starting_node = max(
    pre_world_betweenness_centralities, 
    key=pre_world_betweenness_centralities.get
)

starting_node_biome_id = max(
    pre_biomes_betweenness_centralities, 
    key=pre_biomes_betweenness_centralities.get
)

starting_node_biome = BIOMES[starting_node_biome_id]

In [ ]:
sorted(list(pre_biomes_betweenness_centralities.values()))

In [ ]:
# https://stackoverflow.com/a/3071441/8423001
(
    stats.rankdata(
        a=list(pre_biomes_betweenness_centralities.values()),
        method="dense"
    )
    # Because the ranks start at 1 but Python is 0-indexed,
    # subtract 1.
    - 1
)

In [ ]:
sum(np.array(list(pre_biomes_betweenness_centralities.values())) <= 0.01)

In [ ]:
stats.rankdata(
        a=[-2, 0, 3, 3, 3],
        method="max"
    )

In [ ]:
# def stochastic_func(
#     x,
#     b,
#     corr
# ):
#     rng = np.random.default_rng()
#     std_x = np.std(x)
#     if std_x == 0:
#         y = rng.choice(np.arange(b + 1))
#     else:
#         x_normalized = (x - np.mean(x))/np.std(x)
 
#         y_normalized = corr * x_normalized
#         std_ints = np.std(np.arange(b + 1))
#         mean_ints = (1 + b)/2
#         y = y_normalized * std_ints + mean_ints
#     return y

Copula Stuff

In [ ]:
def gaussian_copula(*args, **kwargs):
    """Get the value of a Gaussian Copula."""
    # https://en.wikipedia.org/wiki/Copula_(probability_theory)#Gaussian_copula
    # Arguments provided via position should be 
    # real numbers in [0, 1].  
    # kwargs should contain a key=value combination
    # where the key is cov.
    #
    # The multivariate_normal.cdf returns nan when the corresponding
    # probability law is at least two dimensional and at least one of 
    # the values supplied to x is -inf.  However, we think that it is
    # reasonable for it just to return 0 instead of nan.
    x = stats.norm.ppf(q=args)
    if (x == float("-inf")).any():
        cdf = 0
    else:
        cdf = stats.multivariate_normal.cdf(
            x=stats.norm.ppf(q=args),
            mean=np.zeros(shape=len(args)),
            allow_singular=True,
            **kwargs      
        )

    return cdf

In [ ]:
def bivariate_discrete_copula_pmf(C, u:int, v:int, R:int, S:int, **kwargs) -> float:
    """Get the value of the probability mass function
    at (u, v) using the copula function C.

    source: https://doi.org/10.1515/demo-2020-0022
    see: equation 7.1
    """
    if (u < 0) or (u > (R - 1)):
        raise ValueError("u must be in {0, 1, ..., R - 1}")
    if (v < 0) or (v > (S - 1)):
        raise ValueError("v must be in {0, 1, ..., S - 1}")

    pmf = C((u + 1)/R, (v + 1)/S, **kwargs) \
        - C(u/R, (v + 1)/S, **kwargs) \
        - C((u + 1)/R, v/S, **kwargs) \
        + C(u/R, v/S, **kwargs)
    
    if (pmf < (0 - sys.float_info.epsilon)) or (pmf > (1 + sys.float_info.epsilon)):
        raise RuntimeError("C appears to be an invalid copula.")
    
    return pmf

In [ ]:
def make_conditional_pmf(C, R:int, S:int, **kwargs):
    """Make conditional PMF array.  
    
    For all u in {0, 1, ..., R - 1},
    determine the conditional distribution:
    P(V=v|U=u).
    Save this as a two-dimensional array
    where the (i, j) entry in the array
    represents P(V=j|U=i).

    Args:
        C: function. This is the function for a copula.

        **kwargs: additional name=value pairs that can
            be passed to C.
  
    Returns:
        numpy.ndarray.    
    """
    
    conditional_pmf_array = np.empty(shape=(R, S))
    for u in range(R):
        for v in range(S):
            # Save a preliminary value.
            conditional_pmf_array[u, v] = bivariate_discrete_copula_pmf(
                C=gaussian_copula, 
                u=u, 
                v=v, 
                R=R, 
                S=S, 
                **kwargs
            )
        # Now, after getting part of the array filled out,
        # do some rescaling to make sure we are
        # constructing a valid probability distribution.
        probs_as_ints = (conditional_pmf_array[u, :] * (2 ** (32 - 1))).astype(np.int32)
        probs_as_probs = (probs_as_ints / probs_as_ints.sum())
        conditional_pmf_array[u, :] = probs_as_probs

    return conditional_pmf_array

In [ ]:
num_biomes = len(BIOMES)
corr = np.array([
    [1, 0.9],
    [0.9, 1]
])

conditional_pmf = make_conditional_pmf(
    C=gaussian_copula,
    R=NUM_WORLD_LOCATIONS,
    S=num_biomes,
    cov=corr
)

In [ ]:
def get_correlated_ranks(
    conditional_pmf,
    rng
):
    """Given a bivariate conditional_pmf formatted
    as an array, return 0-index-based ranks.

    Args:
        conditional_pmf: numpy.ndarray
        rng: numpy.random._generator.Generator
    
    Returns:
        numpy.ndarray. The order of the elements
        in the 1-dimensional array is significant.
    """
    conditional_pmf_shape = conditional_pmf.shape
    num_x_ranks = conditional_pmf_shape[0]
    num_y_ranks = conditional_pmf_shape[1]

    if num_x_ranks < num_y_ranks:
        raise NotImplementedError(
"num_y_ranks must be <= num_x_ranks\n \
Please make sure that conditional_pmf has a \
number of rows greater than or equal to its \
number of columns.  Also, make sure that \
each row is a valid probability distribution."
        )  
      
    y_ranks = np.empty(shape=num_x_ranks, dtype=int)
    
    # Before loop
    is_surjective = False

    # Repeatedly generate possible realizations of 
    # ranks for the Y random variable
    # until surjectivity is achieved.
    while is_surjective is False:
        for x_rank in range(num_x_ranks):
            # Choose y_ranks[x_rank] based on 
            # the conditional PMF for 
            # the current value of x_rank.
            y_ranks[x_rank] = rng.choice(
                # Choose from all of the possible
                # Y ranks.
                a=num_y_ranks, 
                # Weight the choice according to
                # the conditional_pmf.
                p=conditional_pmf[x_rank, :], 
                size=1,
                replace=True,
                shuffle=False
            ).item()

        # Test for surjectivity after building out y_ranks
        is_surjective = bool(
            np.isin(
                element=np.arange(num_y_ranks), 
                test_elements=y_ranks
            ).all()
        )

    return y_ranks

In [ ]:
biome_indices_for_world_locations = get_correlated_ranks(
    conditional_pmf=conditional_pmf,
    rng=rng
)

biomes_for_world_locations = [BIOMES[b] for b in biome_indices_for_world_locations]

In [ ]:
# https://realpython.com/iterate-through-dictionary-python/#iterating-through-dictionaries-comprehension-examples
{n: {"biome": biomes_for_world_locations[n]} for n in range(NUM_WORLD_LOCATIONS)}

In [ ]:
# We plan on assigning biomes to the nodes in our world.
# But, we must consider that some biomes are more likely
# to be connected.  Thus, we assign the biomes randomly
# while taking account of the betweenness centralities.
# With probability 0.5, we assign neighbors the same
# biome, while with probability 0.5, we assign neighbors
# a new biome of similar betweenness centrality.
1.0 / NUM_WORLD_LOCATIONS
sorted(pre_biomes_betweenness_centralities.values())
# Given a value of the ECDF of pre_world_betweenness_centralities
# generate an appropriately positioned random rank
# within pre_biomes_betweenness_centralities.
# First, rank the pre_world_betweenness_centralities.
sorted(pre_world_betweenness_centralities.values())
# Second, find the find the value of the ECDF for each rank.

In [ ]:
np.quantile(
    a=list(pre_world_betweenness_centralities.values()),
    q=0.5
)

In [ ]:
for id in pre_biomes.neighbors(starting_node_biome_id):
    print(id)

In [ ]:
# Loop through nodes and set initial parameters.
for node in nx.nodes(G=pre_world):
    nx.set_node_attributes(
        G=pre_world, 
        # https://realpython.com/iterate-through-dictionary-python/#iterating-through-dictionaries-comprehension-examples
        values={n: {"biome": biomes_for_world_locations[n]} for n in range(NUM_WORLD_LOCATIONS)}
        # {
        #     node: {"carrying_capacity": 1000000},
        # }
    )

In [ ]:
# https://github.com/WestHealth/pyvis/issues/48
world_layout = nx.spring_layout(G=pre_world, iterations=1, threshold=0.01)

In [ ]:
world.from_nx(nx_graph=pre_world, show_edge_weights=True)
for node in world.nodes:
    node["x"] = world_layout[node["id"]][0] * 1000
    node["y"] = world_layout[node["id"]][1] * 1000
world.toggle_physics(False)
world.show("fast_world.html")

In [ ]:
world.from_nx(nx_graph=pre_world, show_edge_weights=True)
world.show("world.html")